# Singapore Flat Resale Prices 1990-Present

- Month - Month of sale
- Type - Designated residential area with its own amenities, infrastructure, and community facilities
- Flat Type - Classification of units by room size. They range from 2 to 5 rooms, 3Gen units, and Executive units.
- Block - A HDB building comprising multiple flats or apartments
- Street Name - Name of the road the HDB flat is located along
- Storey Range - Estimated range of floors the unit sold was located on
- Floor Area - Total interior space within the unit, measured in square meters
- Flat Model - Classification of units by generation of which the flat was made, ranging from New Generation, DBSS, Improved, Apartment
- Lease Commence Date - Starting point of a lease agreement, marking the beginning of the lease term during which the tenant has the right to use and occupy the leased property
- Resale Price - Cost of the flat sold

In [1]:
from src.config import RAW_DATA_DIR, EXTERNAL_DATA_DIR

2025-07-14 22:42:43.022 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /Users/Edward/Documents/GitHub/sg-flat-resale


In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [3]:
raw_data_files = ['ResaleFlatPricesBasedonApprovalDate19901999.csv',
                  'ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv',
                  'ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv',
                  'ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv',
                  'ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv']

dfs = [pd.read_csv(RAW_DATA_DIR / file) for file in raw_data_files]
df = pd.concat(dfs, ignore_index=True)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
941100,2024-11,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,720000.0,91 years 10 months
941101,2024-11,YISHUN,5 ROOM,850,YISHUN ST 81,01 TO 03,122.0,Improved,1988,670000.0,62 years 09 months
941102,2024-11,YISHUN,EXECUTIVE,405,YISHUN AVE 6,04 TO 06,148.0,Maisonette,1988,855500.0,62 years 10 months
941103,2024-11,YISHUN,EXECUTIVE,356,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,930000.0,62 years 10 months


In [4]:
# Combine "MULTI-GENERATION" and "MULTI GENERATION" flat types
df['flat_type'] = df['flat_type'].replace({'MULTI GENERATION' : 'MULTI-GENERATION'})

# Create date column by setting the day from each month to 01 since the original time format provided is has no day provided
df['date'] = pd.to_datetime(df['month'], format='%Y-%m')

# Create month column
df['month'] = df['date'].dt.strftime('%m').astype('int64')

# Create year column
df['year'] = df['date'].dt.strftime('%Y').astype('int64')

# Rename lease_commence_date and add years_leased column
df = df.rename({'lease_commence_date':'lease_year'}, axis=1)
df['years_leased'] = df['year'] - df['lease_year']

# Remove entries where years_leased<0
df = df[df['years_leased']>=0]

# Set entries to proper case
for col in ['town', 'street_name', 'flat_model', 'flat_type']:
    df[col] = df[col].str.title()
df['street_name'] = df['street_name'].replace({"'S":"'s"}, regex=True)

# Split storey_range into 2 columns and calculate number of storeys
df[['start_floor', 'end']] = df['storey_range'].str.extract(r'(\d+)\s+TO\s+(\d+)').astype(int)
df['storey_count'] = df['end'] - df['start_floor']

# Rearrange columns
df = df.loc[:, ['date', 'year', 'month', 'town', 'street_name', 'block', 'flat_type', 'flat_model', 
                'storey_count', 'start_floor', 'floor_area_sqm', 'lease_year', 'years_leased', 'resale_price']]

In [5]:
df.head()

,date,year,month,town,street_name,block,flat_type,flat_model,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price
0,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,9000.0
1,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,4,31.0,1977,13,6000.0
2,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,8000.0
3,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,7,31.0,1977,13,6000.0
4,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,216,3 Room,New Generation,2,4,73.0,1976,14,47200.0


The data source does not indicate whether the resale prices are adjusted for inflation. It may be necessary to adjust the prices for inflation.

In [6]:
infl = pd.read_csv(EXTERNAL_DATA_DIR / 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_77.csv')
infl

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.421441,0.474764,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.370290,5.245878,6.571396,6.399343,4.720805,4.653665,5.405162,7.240978,10.773751,7.126975
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.673996,-0.661709,4.383892,4.975952,0.626149,2.302373,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.768436,2.130817,1.487416,1.725486,1.784050,1.760112,2.437609,3.653533,7.967574,4.670084
4,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,7.280387,9.355972,30.694415,29.844480,19.628938,17.080954,22.271539,25.754295,21.355290,13.644102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.428958,-0.536929,0.273169,1.488234,1.053798,2.675992,0.198228,3.353691,11.580510,4.944227
262,"Yemen, Rep.",YEM,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,8.104726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1.288859,2.102374,1.246285,1.33797,2.534973,4.069029,...,6.129838,4.540642,6.571396,5.184247,4.517165,4.120246,3.210036,4.611672,7.039727,6.073909
264,Zambia,ZMB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,7.806876,10.110593,17.869730,6.577312,7.494572,9.150316,15.733060,22.020768,10.993204,10.884532


In [7]:
# Create dataframe for percent inflation figures in Singapore
infl = pd.read_csv(EXTERNAL_DATA_DIR / 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_77.csv')
infl = infl.loc[infl['Country Name'] == 'Singapore', '1990':]
infl = infl.rename({208:'inflation%'})

# Add 2024 inflation as 2.4
infl['2024'] = 2.389511236

# Transpose inflation dataframe
infl = infl.T

# Add calculations for point indexes
infl['point_index'] = infl['inflation%'].apply(lambda a: 1+a/100)

# Calculate cumulative inflation figures (2024)
infl['cum_index'] = infl['point_index'].iloc[::-1].cumprod()

# Reset index, rename index column to 'year', set the dtype to int64
infl = infl.reset_index()
infl = infl.rename({'index':'year'}, axis=1)
infl['year'] = infl['year'].astype('int64')

infl

,year,inflation%,point_index,cum_index
0,1990,3.460753,1.034608,1.916950
1,1991,3.425702,1.034257,1.852828
2,1992,2.263071,1.022631,1.791458
3,1993,2.289300,1.022893,1.751813
4,1994,3.100133,1.031001,1.712607
5,1995,1.720534,1.017205,1.661110
6,1996,1.383181,1.013832,1.633013
7,1997,2.003586,1.020036,1.610734
8,1998,-0.267502,0.997325,1.579096
9,1999,0.016710,1.000167,1.583331


In [8]:
# Merge original dataframe with the inflation dataframe on year
df = pd.merge(df, infl[['year', 'cum_index']], on='year', how='left')

# Add column for resale price adjusted by inflation
df['infl_adj_price'] = df['resale_price']*df['cum_index']
df['infl_adj_price'] = df['infl_adj_price'].round(1)
df = df.drop('cum_index', axis=1)
df.head()

,date,year,month,town,street_name,block,flat_type,flat_model,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price,infl_adj_price
0,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,9000.0,17252.5
1,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,4,31.0,1977,13,6000.0,11501.7
2,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,8000.0,15335.6
3,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,7,31.0,1977,13,6000.0,11501.7
4,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,216,3 Room,New Generation,2,4,73.0,1976,14,47200.0,90480.0


There is now a column for the resale price of each unit adjusted for inflation (2024)  
Source of inflation figures: World Bank (https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?contextual=default&end=2023&locations=SG&start=1961&view=chart)

In [9]:
df.describe()

,date,year,month,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price,infl_adj_price
count,941054,941054.000000,941054.000000,941054.000000,941054.000000,941054.000000,941054.000000,941054.000000,9.410540e+05,9.410540e+05
mean,2006-11-24 03:44:27.251825920,2006.433233,6.577723,2.014533,6.697483,95.685103,1988.421720,18.011514,3.252501e+05,4.419170e+05
min,1990-01-01 00:00:00,1990.000000,1.000000,2.000000,1.000000,28.000000,1966.000000,0.000000,5.000000e+03,9.584700e+03
25%,1999-02-01 00:00:00,1999.000000,4.000000,2.000000,4.000000,73.000000,1981.000000,9.000000,1.950000e+05,3.009650e+05
50%,2005-07-01 00:00:00,2005.000000,7.000000,2.000000,7.000000,93.000000,1986.000000,16.000000,3.000000e+05,4.210787e+05
75%,2014-10-01 00:00:00,2014.000000,10.000000,2.000000,10.000000,113.000000,1996.000000,25.000000,4.220000e+05,5.582552e+05
max,2024-11-01 00:00:00,2024.000000,12.000000,4.000000,49.000000,366.700000,2021.000000,58.000000,1.588000e+06,1.625945e+06
std,NaN,9.498893,3.401196,0.169865,4.841773,25.800485,10.809216,10.835797,1.746735e+05,1.969821e+05


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941054 entries, 0 to 941053
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            941054 non-null  datetime64[ns]
 1   year            941054 non-null  int64         
 2   month           941054 non-null  int64         
 3   town            941054 non-null  object        
 4   street_name     941054 non-null  object        
 5   block           941054 non-null  object        
 6   flat_type       941054 non-null  object        
 7   flat_model      941054 non-null  object        
 8   storey_count    941054 non-null  int64         
 9   start_floor     941054 non-null  int64         
 10  floor_area_sqm  941054 non-null  float64       
 11  lease_year      941054 non-null  int64         
 12  years_leased    941054 non-null  int64         
 13  resale_price    941054 non-null  float64       
 14  infl_adj_price  941054 non-null  flo

In [11]:
town_geo = gpd.read_file(EXTERNAL_DATA_DIR / 'district_and_planning_area.geojson')
town_geo.sort_values(by=['district', 'planning_area'])

,district,planning_area,geometry
28,Central,Bishan,"MULTIPOLYGON (((103.84297 1.36429, 103.84297 1..."
3,Central,Bukit Merah,"MULTIPOLYGON (((103.82362 1.26018, 103.82362 1..."
1,Central,Bukit Timah,"MULTIPOLYGON (((103.79766 1.34813, 103.79806 1..."
5,Central,Downtown Core,"MULTIPOLYGON (((103.86655 1.30386, 103.86696 1..."
30,Central,Geylang,"MULTIPOLYGON (((103.90179 1.30974, 103.9015 1...."
36,Central,Kallang,"MULTIPOLYGON (((103.86277 1.3303, 103.86302 1...."
39,Central,Marina East,"MULTIPOLYGON (((103.88025 1.28386, 103.88024 1..."
40,Central,Marina South,"MULTIPOLYGON (((103.86473 1.28885, 103.86909 1..."
12,Central,Marine Parade,"MULTIPOLYGON (((103.90806 1.30982, 103.90822 1..."
41,Central,Museum,"MULTIPOLYGON (((103.84575 1.30092, 103.8464 1...."


In [12]:
# Add planning area column for geospatial data analysis
df['planning_area'] = df['town']
df['planning_area'].unique()

array(['Ang Mo Kio', 'Bedok', 'Bishan', 'Bukit Batok', 'Bukit Merah',
       'Bukit Timah', 'Central Area', 'Choa Chu Kang', 'Clementi',
       'Geylang', 'Hougang', 'Jurong East', 'Jurong West',
       'Kallang/Whampoa', 'Marine Parade', 'Queenstown', 'Sengkang',
       'Serangoon', 'Tampines', 'Toa Payoh', 'Woodlands', 'Yishun',
       'Lim Chu Kang', 'Sembawang', 'Bukit Panjang', 'Pasir Ris',
       'Punggol'], dtype=object)

In [13]:
# Modify planning_area entries for town == Central Area and town == Kallang/Whampoa
town_mappings = {
    "Central Area": {
        "Outram": ["Outram", "Smith St", "Jln Kukoh", "Sago Lane", "New Mkt Rd", 
                   "Upp Cross St", "Chin Swee Rd", "Kreta Ayer Rd", "Cantonment Rd"],
        "Rochor": ["Queen", "Rowell", "Rochor", "Bain St", "Short St", "Jln Berseh", 
                   "Selegie Rd", "Buffalo Rd", "Chander Rd", "Klang Lane", "Kelantan Rd", 
                   "Waterloo St", "Veerasamy Rd"],
        "Bukit Merah": ["Tg Pagar Plaza"]
    },
    "Kallang/Whampoa": {
        "Novena": ["Whampoa", "Kent Rd", "Jln Rajah", "Lor Limau", "Jln Dusun", 
                   "Ah Hood Rd", "Moulmein Rd", "Jln Bahagia", "Jln Tenteram", "Gloucester Rd"],
        "Kallang": ["Owen Rd", "Jln Batu", "Mcnair Rd", "Towner Rd", "Dorset Rd", "French Rd",
                    "Jln Ma'Mor", "Kg Kayu Rd", "Kg Arang Rd", "Jellicoe Rd", "Lor 3 Geylang",
                    "Tessensohn Rd", "Farrer Pk Rd", "Boon Keng Rd", "Bendemeer Rd", "Cambridge Rd",
                    "Crawford Lane", "Nth Bridge Rd", "Geylang Bahru", "Kallang Bahru", "Race Course Rd",
                    "St. George's Rd", "Upp Boon Keng Rd", "St. George's Lane", "King George's Ave"]
    }
}

# Apply street mappings with town filter
def apply_street_mappings(df, mappings):
    for town, areas in mappings.items():
        town_mask = df['town'] == town
        for area, streets in areas.items():
            pattern = '|'.join(streets)
            street_mask = df['street_name'].str.contains(pattern, case=False, na=False)
            df.loc[town_mask & street_mask, 'planning_area'] = area

apply_street_mappings(df, town_mappings)

# Handle specific block mappings
beach_rd_blocks = {
    'Rochor': ['1', '2', '3', '6'],
    'Kallang': ['15', '17']
}

beach_rd_mask = (df['town'] == 'Kallang/Whampoa') & (df['street_name'] == 'Beach Rd')
for area, blocks in beach_rd_blocks.items():
    block_pattern = '|'.join(blocks)
    df.loc[beach_rd_mask & df['block'].str.contains(block_pattern, case=False, na=False), 'planning_area'] = area

In [14]:
# Import geojson file
town_geo = gpd.read_file(EXTERNAL_DATA_DIR / 'district_and_planning_area.geojson')

# Merge geojson with df_town
df = pd.merge(df, town_geo, on='planning_area', how='left')

df

,date,year,month,town,street_name,block,flat_type,flat_model,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price,infl_adj_price,planning_area,district,geometry
0,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,9000.0,17252.5,Ang Mo Kio,North-East,"MULTIPOLYGON (((103.85721 1.39654, 103.85731 1..."
1,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,4,31.0,1977,13,6000.0,11501.7,Ang Mo Kio,North-East,"MULTIPOLYGON (((103.85721 1.39654, 103.85731 1..."
2,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,8000.0,15335.6,Ang Mo Kio,North-East,"MULTIPOLYGON (((103.85721 1.39654, 103.85731 1..."
3,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,7,31.0,1977,13,6000.0,11501.7,Ang Mo Kio,North-East,"MULTIPOLYGON (((103.85721 1.39654, 103.85731 1..."
4,1990-01-01,1990,1,Ang Mo Kio,Ang Mo Kio Ave 1,216,3 Room,New Generation,2,4,73.0,1976,14,47200.0,90480.0,Ang Mo Kio,North-East,"MULTIPOLYGON (((103.85721 1.39654, 103.85731 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941049,2024-11-01,2024,11,Yishun,Yishun St 51,511B,5 Room,Improved,2,10,113.0,2017,7,720000.0,737204.5,Yishun,North,"MULTIPOLYGON (((103.85955 1.42552, 103.86197 1..."
941050,2024-11-01,2024,11,Yishun,Yishun St 81,850,5 Room,Improved,2,1,122.0,1988,36,670000.0,686009.7,Yishun,North,"MULTIPOLYGON (((103.85955 1.42552, 103.86197 1..."
941051,2024-11-01,2024,11,Yishun,Yishun Ave 6,405,Executive,Maisonette,2,4,148.0,1988,36,855500.0,875942.3,Yishun,North,"MULTIPOLYGON (((103.85955 1.42552, 103.86197 1..."
941052,2024-11-01,2024,11,Yishun,Yishun Ring Rd,356,Executive,Maisonette,2,1,146.0,1988,36,930000.0,952222.5,Yishun,North,"MULTIPOLYGON (((103.85955 1.42552, 103.86197 1..."


In [15]:
df_town = df.groupby(['district', 'planning_area', 'geometry', 'year'], group_keys=False)['infl_adj_price'].mean().reset_index(name='avg_price')
df_town

,district,planning_area,geometry,year,avg_price
0,Central,Bishan,"MULTIPOLYGON (((103.84297 1.36429, 103.84297 1...",1990,177773.583962
1,Central,Bishan,"MULTIPOLYGON (((103.84297 1.36429, 103.84297 1...",1991,208801.046094
2,Central,Bishan,"MULTIPOLYGON (((103.84297 1.36429, 103.84297 1...",1992,271667.549598
3,Central,Bishan,"MULTIPOLYGON (((103.84297 1.36429, 103.84297 1...",1993,389694.777642
4,Central,Bishan,"MULTIPOLYGON (((103.84297 1.36429, 103.84297 1...",1994,459143.989104
...,...,...,...,...,...
963,West,Jurong West,"MULTIPOLYGON (((103.72813 1.34888, 103.72816 1...",2020,480262.975747
964,West,Jurong West,"MULTIPOLYGON (((103.72813 1.34888, 103.72816 1...",2021,531142.168400
965,West,Jurong West,"MULTIPOLYGON (((103.72813 1.34888, 103.72816 1...",2022,568863.936616
966,West,Jurong West,"MULTIPOLYGON (((103.72813 1.34888, 103.72816 1...",2023,547489.375464
